In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Flatten
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Input, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout


ecom=pd.read_csv('Womens Clothing E-Commerce Reviews.csv')


ecom.head()

ecom.describe()

ecom.shape

ecomc=ecom.copy()

ecomc['Title']=ecomc['Title'].fillna('no title')
ecomc['Review Text']=ecomc['Review Text'].fillna('no review')
ecomc['Class Name']=ecomc.groupby('Department Name')['Class Name'].transform(lambda x:x.fillna(x.mode()[0]))
ecomc.drop(['Unnamed: 0','Clothing ID','Department Name','Division Name'],axis=1,inplace=True)


vecab_size=1000
max_len=100
combined_text = ecomc['Title'].fillna('no title') + ' ' + ecomc['Review Text'].fillna('no review')

tokinizer=Tokenizer(num_words=vecab_size)
tokinizer.fit_on_texts(combined_text)

X_text=tokinizer.texts_to_sequences(combined_text)
X_text=pad_sequences(X_text,maxlen=max_len,padding='post')


ecomc.info()

# OP

# ecomc.drop(['Unnamed: 0','Clothing ID','Department Name','Division Name'],axis=1,inplace=True)

# ecomc['Division Name'].fillna(ecomc['Division Name'].mode()[0])

# ecomc['Department Name']=ecomc.groupby('Class Name')['Department Name'].transform(lambda x:x.fillna(x.mode()[0]))

ecomc=pd.get_dummies(ecomc,columns=['Class Name'],drop_first=True)

for col in ecomc.select_dtypes(include='bool').columns:
    ecomc[col]=ecomc[col].map({True:1,False:0})
    

# X_cat=ecomc[['Class Name_Casual bottoms','Class Name_Chemises','Class Name_Dresses','Class Name_Fine gauge','Class Name_Intimates','Class Name_Jackets','Class Name_Jeans','Class Name_Knits','Class Name_Layering','Class Name_Lounge','Class Name_Outerwear','Class Name_Pants','Class Name_Shorts','Class Name_Skirts','Class Name_Sleep','Class Name_Sweaters','Class Name_Swim','Class Name_Trend']]
X_cat=ecomc.filter(like='Class Name_')
# SCALLING
scaler=StandardScaler()
ecomc['Age']=scaler.fit_transform(ecomc[['Age']])
ecomc['Positive Feedback Count']=scaler.fit_transform(ecomc[['Positive Feedback Count']])

X_num=ecomc[['Age','Positive Feedback Count']]

X_tabular=np.hstack([X_num,X_cat])

# preparing target

y=ecomc['Rating'].values-1

# SPLIT&TRAIN
X_text_train,X_text_test,X_tab_train,X_tab_test,y_train,y_test=train_test_split(X_text,X_tabular,y,test_size=0.2,train_size=0.8,random_state=100)


text_input=Input(shape=(max_len,))
embedding=Embedding(input_dim=vecab_size,output_dim=64,input_length=max_len)(text_input)
lstm=LSTM(64,return_sequences=True)(embedding)
cnn=Conv1D(64,kernel_size=3,activation='relu')(lstm)
pool=GlobalMaxPooling1D()(cnn)

tabular_input=Input(shape=(X_tabular.shape[1],))
dense_tab=Dense(32,activation='relu')(tabular_input)

# merge
merged=Concatenate()([dense_tab,pool])
dense1=Dense(64,activation='relu')(merged)
drop1=Dropout(0.5)(dense1)
output = Dense(5, activation='softmax')(drop1)





model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(
    [X_text_train, X_tab_train], y_train,
    validation_data=([X_text_test, X_tab_test], y_test),
    epochs=10,
    batch_size=64
)


